### Model for the question and answers:

In [1]:
%load_ext autoreload
%autoreload 2
from mc_challenge.dataset    import Questions, Section, load_mc, convert_to_idx
from mc_challenge.validation import validate
from mc_challenge.vocab      import Vocab


training, validation, test = load_mc("data/", 0.1)

Collect the vocabulary from each of the sections of the dataset:

In [3]:
section_vocab = Vocab()
for section in (training + validation + test):
    section_vocab.add_words_from_text(section.text, tokenization = True)
    for q in section.questions:
        section_vocab.add_words_from_text(q.question, tokenization = True)
        for a in q.answers:
            section_vocab.add_words_from_text(a, tokenization = True)

Find the words that only appear in the questions and answers:

In [4]:
section_vocab_no_q = Vocab()
for section in (training + validation + test):
    section_vocab_no_q.add_words_from_text(section.text, tokenization=True)
delta_voc = section_vocab - section_vocab_no_q
delta_voc.word_occ.most_common(10)

[('Who', 328),
 ('Which', 78),
 ('Blue', 35),
 ('Green', 29),
 ('Play', 27),
 ('White', 26),
 ('4', 24),
 ('An', 24),
 ('Yellow', 23),
 ('Black', 23)]

Now that we have a vocabulary we convert the sections to numbers:

In [4]:
training_idx   = convert_to_idx(training, section_vocab)
validation_idx = convert_to_idx(validation, section_vocab)
test_idx       = convert_to_idx(test, section_vocab)

First approach, look at word embeddings in quadratic form model:

In [5]:
from mc_challenge.simple_qform_model import QFormModel
from mc_challenge.dragon_model import DragonModel
import numpy as np


In [48]:
hidden_size = 30
num_answers = 4
vocab_size  = len(section_vocab)

#model = QFormModel(hidden_size, vocab_size, num_answers)
model = DragonModel(hidden_size=100,
                    internal_features=100,
                    intermediate_size=100,
                    vocab_size=vocab_size,
                    num_answers=num_answers,
                    tensor=False,
                    method="sgd")

In [55]:
model.lr.set_value(0.002)

Some test data:

In [40]:
np_sentence  = section_vocab(training[0].text,                    True)
np_question  = section_vocab(training[0].questions[0].question,   True)
np_answer1   = section_vocab(training[0].questions[0].answers[0], True)
np_answer2   = section_vocab(training[0].questions[0].answers[1], True)
np_answer3   = section_vocab(training[0].questions[0].answers[2], True)
np_answer4   = section_vocab(training[0].questions[0].answers[3], True)

In [41]:
model.error_fun(
    np_sentence,
    np_question,
    np_answer1,
    1.0,
    np_answer2,
    0.0,
    np_answer3,
    0.0,
    np_answer4,
    0.0)

array(2.8262931067785217)

In [24]:

def test_score(model):
    return validate(test, model, section_vocab)

In [51]:
test_score(model)

0.3107142857142857

In [56]:
#for section in range(section
num_seen = 0
minibatch_size = 200
max_epochs = 200
error = 0.0
training_order = np.arange(0, len(training_idx))
for epoch in range(max_epochs):
    np.random.shuffle(training_order)
    for section_i in training_order:
        section = training_idx[section_i]
        for q in section.questions:
            q_idx, ans_idx, correct_ans = q
            error += model.update_gradient(
                section.text,
                q_idx,
                ans_idx[0],
                1.0 if correct_ans == 0 else 0.0,
                ans_idx[1],
                1.0 if correct_ans == 1 else 0.0,
                ans_idx[2],
                1.0 if correct_ans == 2 else 0.0,
                ans_idx[3],
                1.0 if correct_ans == 3 else 0.0)
            num_seen += 1
            if (num_seen % minibatch_size) == 0:
                model.apply_gradient()
                val_error = validate(validation, model, section_vocab) * 100.0
                
                print("epoch %d: error validation accuracy %.2f%%, training error %.5f" % (epoch, val_error, error / minibatch_size))
                error = 0.0
                
            #if (num_seen % (10 * minibatch_size)) == 0:
            #    model.reset_caches()

epoch 0: error validation accuracy 32.78%, training error 2.20327
epoch 0: error validation accuracy 30.56%, training error 2.17825
epoch 0: error validation accuracy 32.22%, training error 2.19355
epoch 0: error validation accuracy 32.22%, training error 2.17944
epoch 0: error validation accuracy 34.44%, training error 2.16054
epoch 0: error validation accuracy 32.78%, training error 2.17995
epoch 0: error validation accuracy 32.22%, training error 2.18094
epoch 0: error validation accuracy 31.11%, training error 2.17573
epoch 1: error validation accuracy 31.11%, training error 2.18240
epoch 1: error validation accuracy 31.11%, training error 2.17901
epoch 1: error validation accuracy 31.67%, training error 2.18629
epoch 1: error validation accuracy 31.67%, training error 2.19333
epoch 1: error validation accuracy 32.78%, training error 2.17707
epoch 1: error validation accuracy 31.67%, training error 2.17446
epoch 1: error validation accuracy 32.78%, training error 2.16149
epoch 1: e

KeyboardInterrupt: 